In [1]:
import pandas as pd
import seoapikits

In [2]:
# https://cdn.dataforseo.com/v3/examples/python/python_Client.zip
keywords_list = pd.read_excel('./sample_data/await_finding_url.xlsx', engine='openpyxl')['company_name_cv'].to_list()[:5]

We read the sample data, we only pick the first 5, then Initialize the cls to search for

In [3]:
username = input("Enter the username: ")
password = input("Enter the password: ")
search_api_cls = seoapikits.SERP.SERPAPI(username=username, password=password)

After, we try to post the data, which means we send request to the seodata server, and we can get the task id, Then we could directly got the result by using get. However if request "get" so fast it could not finish search and cast error, so we can use timeout limit to make it to retry. default freq is 5 seconds and timeout is 0 seconds, However you can reset it to make better usage, here is the example

In [4]:
task_post_df = search_api_cls.tasks_seq_post(keyword_list=keywords_list,res_type='df')
task_post_df

[[{'language_code': 'en', 'location_code': 2840, 'keyword': '"Johnnie" Johnson Housing Trust Ltd.'}, {'language_code': 'en', 'location_code': 2840, 'keyword': '051 Subs'}, {'language_code': 'en', 'location_code': 2840, 'keyword': '063 Loan Company Limited'}, {'language_code': 'en', 'location_code': 2840, 'keyword': '0880 Santander Santibañez Sl'}, {'language_code': 'en', 'location_code': 2840, 'keyword': '0km Finance'}]]


,id,keyword
0,09201215-6469-0066-0000-406e5940320a,"""Johnnie"" Johnson Housing Trust Ltd."
1,09201215-6469-0066-0000-67e1c270120a,051 Subs
2,09201215-6469-0066-0000-35a249147847,063 Loan Company Limited
3,09201215-6469-0066-0000-aa0a50c991ee,0880 Santander Santibañez Sl
4,09201215-6469-0066-0000-83d5a2bdf605,0km Finance


In [5]:
task_post_df

,id,keyword
0,09201215-6469-0066-0000-406e5940320a,"""Johnnie"" Johnson Housing Trust Ltd."
1,09201215-6469-0066-0000-67e1c270120a,051 Subs
2,09201215-6469-0066-0000-35a249147847,063 Loan Company Limited
3,09201215-6469-0066-0000-aa0a50c991ee,0880 Santander Santibañez Sl
4,09201215-6469-0066-0000-83d5a2bdf605,0km Finance


In [6]:
task_get_df = search_api_cls.tasks_seq_get_df(task_post_df['id'], timeout_limit=0)
task_get_df

  0%|          | 0/5 [00:00<?, ?it/s]


ConnectionError: Error-timeout for not get a efficient response from Seodata

You can see, so fast search will result for not enough time for server to response. So we take time to retry.
More easily, you could use ```search_api_cls.task_ready``` to checkin

Wait for a while and check again. We could have a simple method which have a while to wait for all task finished

In [10]:
task_ready_df = search_api_cls.tasks_seq_ready_df(task_post_df['id'])
task_ready_df

,id,ready
0,09201215-6469-0066-0000-406e5940320a,True
1,09201215-6469-0066-0000-67e1c270120a,True
2,09201215-6469-0066-0000-35a249147847,True
3,09201215-6469-0066-0000-aa0a50c991ee,True
4,09201215-6469-0066-0000-83d5a2bdf605,True


In [11]:
task_get_df = search_api_cls.tasks_seq_get_df(task_post_df['id'], timeout_limit=30)

100%|██████████| 5/5 [00:05<00:00,  1.16s/it]


In [12]:
pd.merge(task_get_df, task_post_df, on='id', validate='many_to_one')

,type,rank_group,rank_absolute,domain,title,description,url,breadcrumb,id,keyword
0,organic,1,1,www.jjhousing.co.uk,Johnnie Johnson Housing: Housing Association,We are Johnnie Johnson Housing ... We are a no...,https://www.jjhousing.co.uk/,https://www.jjhousing.co.uk,09201215-6469-0066-0000-406e5940320a,"""Johnnie"" Johnson Housing Trust Ltd."
1,organic,2,2,uk.linkedin.com,Johnnie Johnson Housing,Johnnie Johnson Housing is a not-for-profit ho...,https://uk.linkedin.com/company/johnniejohnson...,https://uk.linkedin.com › company › johnniejoh...,09201215-6469-0066-0000-406e5940320a,"""Johnnie"" Johnson Housing Trust Ltd."
2,organic,3,4,www.findahomejjhousing.co.uk,Login - JJ Housing,Johnnie Johnson Housing Limited is a Registere...,https://www.findahomejjhousing.co.uk/login/,https://www.findahomejjhousing.co.uk › login,09201215-6469-0066-0000-406e5940320a,"""Johnnie"" Johnson Housing Trust Ltd."
3,organic,4,5,www.facebook.com,Johnnie Johnson Housing | Stockport,"Johnnie Johnson Housing, Cheadle Hulme. 1265 l...",https://www.facebook.com/jjhousingtrust/,https://www.facebook.com › jjhousingtrust,09201215-6469-0066-0000-406e5940320a,"""Johnnie"" Johnson Housing Trust Ltd."
4,organic,5,6,find-and-update.company-information.service.go...,"""johnnie"" johnson housing trust limited - Comp...","""JOHNNIE"" JOHNSON HOUSING TRUST LIMITED - Free...",https://find-and-update.company-information.se...,https://find-and-update.company-information.se...,09201215-6469-0066-0000-406e5940320a,"""Johnnie"" Johnson Housing Trust Ltd."
...,...,...,...,...,...,...,...,...,...,...
383,organic,95,96,www.reezocar.com,"New/0km Fiat Fiorino ad certified : Year 2022,...",Financing. How does it work? Choose your car. ...,https://www.reezocar.com/en/neuve/annonce-fiat...,https://www.reezocar.com › ... › Fiat › Fiat F...,09201215-6469-0066-0000-83d5a2bdf605,0km Finance
384,organic,96,97,www.zigwheels.com,How much we can still drive Skoda Octavia when...,How much we can still drive Skoda Octavia when...,https://www.zigwheels.com/newcars/faqs/how-muc...,https://www.zigwheels.com › newcars › faqs › h...,09201215-6469-0066-0000-83d5a2bdf605,0km Finance
385,organic,97,98,en.nissan-dubai.com,NISSAN Certified Pre-Owned Cars | NISSAN Dubai,Apply for Finance · Finance Calculator · Book ...,https://en.nissan-dubai.com/nissan-certified.html,https://en.nissan-dubai.com › nissan-certified,09201215-6469-0066-0000-83d5a2bdf605,0km Finance
386,organic,98,99,www.quora.com,How far can a car go when it says 0 km to empty?,Profile photo for David Johansson. David Johan...,https://www.quora.com/How-far-can-a-car-go-whe...,https://www.quora.com › How-far-can-a-car-go-w...,09201215-6469-0066-0000-83d5a2bdf605,0km Finance
